In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [90]:
data_path = os.getcwd() + "/dataset/total_test.csv"
# features = ["A_F", "A_V", "A_L", "L_F", "L_T", "L_L", "T_F", "T_V", "T_L"]

dataset = pd.read_csv(data_path)
time = list(dataset['time'])
Action = list(dataset['Action'])

TFdata = dataset[["T_F"]].values
TVdata = dataset[["T_V"]].values
TLdata = dataset[["T_L"]].values


dataframe = dataset[["T_F", "T_V", "T_L"]].values
dataset = dataset[["T_F", "T_V", "T_L", "Action"]].values

window_length = int(1*64)
total_windows = int((len(dataset))/window_length)

In [91]:
TFdata = np.array(TFdata)
TVdata = np.array(TVdata)
TLdata = np.array(TLdata)

In [92]:
Tsensor = np.multiply(TFdata, TFdata) + np.multiply(TVdata, TVdata) + np.multiply(TLdata, TLdata)
Tsensor = np.sqrt(Tsensor)

In [93]:
dataframe = (dataframe-dataframe.mean())/dataframe.std()
Tsensor = (Tsensor-Tsensor.mean())/Tsensor.std()

In [94]:
indices = time
groups = []
group_count = 0
temp = []
lenOfGroup = []
length_count = 0

for i in range(len(indices)):
    if i == (len(indices) - 1):
        temp.append(indices[i])
        length_count = length_count + 1

        groups.append(temp)
        lenOfGroup.append(length_count)
        length_count = 0
        temp = []
        break
    temp.append(indices[i])
    length_count = length_count + 1
    if (indices[i+1] - 20 > indices[i]):
        group_count = group_count + 1

        lenOfGroup.append(length_count)
        length_count = 0

        groups.append(temp)
        temp = []

del temp, indices, group_count, length_count, i

In [95]:
countOfUndivisible = 0
total_windows_with_overlap = 0

start_index = 0

stop_Indexs = []
stop_Index = -window_length


for lengths in lenOfGroup:
    stop_Index = stop_Index + lengths
    stop_Indexs.append(stop_Index)
    total_windows_with_overlap = total_windows_with_overlap + int(lengths/window_length*2 -1)
    
    if lengths % (window_length/2) != 0:
        countOfUndivisible = countOfUndivisible + 1
        print(lengths)

print(countOfUndivisible)
del countOfUndivisible, lengths

0


In [96]:
X_ = np.empty((total_windows_with_overlap, window_length))
y_ = np.empty((total_windows_with_overlap, 1))

stop_Index = 0
window_count = 0

for window in range(total_windows_with_overlap):
    for i in range(window_length):
        if i == 0:
            y_[window] = dataset[int(window_count*window_length), 3]
        
        if int(window_count*window_length)<len(dataset)-window_length-1:
            X_[window, i] = TFdata[int(window_count*window_length) + i]
        
        if stop_Index < len(stop_Indexs):
            if int(window_count*window_length) == stop_Indexs[stop_Index]:
                window_count = window_count + 0.5
                stop_Index = stop_Index + 1

        if window == total_windows_with_overlap-1:
            X_[window, i] = TFdata[int((window_count-0.5)*window_length) + i]
    window_count = window_count + 0.5

In [97]:
from scipy.signal import stft
import numpy as np
rng = np.random.default_rng()

In [102]:
save_path = os.getcwd() + "/dataset/STFT_test/"
amp = 20
vmin = 0
vmax = 1

for i in range(total_windows_with_overlap):
    final_path = save_path + str(int(y_[i]))
    x=X_[i].flatten()

    fig = plt.figure(figsize = (1, 1))
    f, t, Zxx = stft(x=x, fs = window_length)
    ax = fig.add_subplot()
    ax = plt.pcolormesh(t, f, np.abs(Zxx), shading='gouraud')
    plt.axis('off')
    # plt.title('STFT Magnitude')
    # plt.ylabel('Frequency [Hz]')
    # plt.xlabel('Time [sec]')
    image_path = final_path + '/' + str(i) + '.png'
    plt.savefig(image_path, bbox_inches='tight',pad_inches = 0)
    plt.close(fig)

In [88]:
fs = 64
N = len(y)
amp = 1
time = np.arange(N) / float(fs)
plt.plot(time, y)

NameError: name 'y' is not defined